# Region Growing

Starting from seed points, the region-growing algorithm is used to decide whether a point should be included in a region or not. Two different implementations are explained in this notebook:
- Region growing based on the Point Cloud Library (PCL) implementation
- Clustering based region growing using "Label Connected Components"


In [ ]:
# Add project src to path.
import set_path

import numpy as np

import src.region_growing as growing
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.math_utils as math_utils
from src.region_growing.region_growing import RegionGrowing
from src.utils.labels import Labels
from src.pipeline import Pipeline

In [ ]:
# Select the file to process. The outfile can be set to 'None' to overwrite the file.
tilecode = '2386_9702'

in_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'
out_file = '../datasets/pointcloud/labelled_' + tilecode + '.laz'

# In this example we want to grow the building points, the initial seed points.
# The ground points must be removed to fast and properly cluster the cloud.
exclude_labels = (Labels.GROUND,)

## Region growing segmentation based on PCL

### Algorithm explained
Based on: https://pcl.readthedocs.io/projects/tutorials/en/latest/region_growing_segmentation.html

With the PCL implementation, a point is picked with minimum curvature value and the region growth process is started. In our implementation, we start from initial seed points that are considered to be inside the object to be segmented. 

For every point in the seed list, the algorithm finds neighboring points. Every neighbor is tested for the angle between its normal and normal of the current seed point. If the angle is less than the threshold value then current point is added to the current region.

In [ ]:
# Region growing with building points as initial seed points. 
region_growing_pcl = RegionGrowing(label=Labels.BUILDING, exclude_labels=exclude_labels)

## Clustering based region growing segmentation

### Algorithm explained
This code performs clustering based region growing segmentation. The main tasks include: 
- The [Label Connected Components Segmentation](https://www.cloudcompare.org/doc/wiki/index.php?title=Label_Connected_Components) algorithm clusters the selected cloud(s) in smaller parts separated by a minimum distance.
- Next, our code grows regions, or in this case all points inside a clusters, based on a percentage of initial seed points inside a cluster. 

### Installation instructions
A Python wrapper provided for CloudCompare allows to use “Label Connected Components”. The Python wrapper works on **linux only**. This wrapper and all the dependencies can be installed using the build and installation instructions in the README at [CloudCompare-PythonPlugin.](https://github.com/tmontaigu/CloudCompare-PythonPlugin.git)

In [ ]:
# Region growing with building points as initial seed points. 
region_growing_clustering = growing.LabelConnectedComp(label=Labels.BUILDING,
                                                       grid_size=0.05,
                                                       min_component_size=100)

### Layered LCC region growing

Sometimes we want to use different parameters for region growing, depending on the height above ground level. For example, close to the ground we expect more clutter, so a more fine-grained approach might be needed, while higher up we can use coarsed clusters.

For this, we can use a LayerLCC region grower, which accepts different parameters for different 'layers' of the point cloud. Each layer has the following (default) parameters:

```
params = {'bottom': -inf,        # Height above ground at which the layer starts.
          'top': inf,            # Height above ground at which the layer stops.
          'grid_size': 0.1       # Octree grid size for LCC method, in meters. Lower means a more fine-grained clustering.
          'min_comp_size': 100,  # Minimum number of points for a component to be considered.
          'threshold': 0.5}      # Minimum fraction of points in a component that are already 
                                 # labelled initially for the component to be added.
```

In [ ]:
building_top = {'bottom': 12., 'grid_size': 0.1, 'threshold': 0.5}
building_bottom = {'bottom': 0.5, 'top': 12., 'grid_size': 0.05, 'threshold': 0.5}

# We need AHN elevation data to determine the height above ground.
ahn_data_folder = '../datasets/ahn/'
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

layered_region_growing = growing.LayerLCC(Labels.BUILDING, ahn_reader, 
                                          params=[building_top, building_bottom])

## Set-up pipeline

In [ ]:
# Set-up region growing based on PCL pipeline.
process_sequence = (region_growing_pcl,)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

In [ ]:
# Set-up clustering based region growing pipeline.
process_sequence = (region_growing_clustering,)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

In [ ]:
# Set-up layer based region growing pipeline.
process_sequence = (layered_region_growing,)
pipeline = Pipeline(processors=process_sequence, exclude_labels=exclude_labels, caching=False)

### Process the LAS file

In [ ]:
pipeline.process_file(in_file, out_file=out_file)